In [1]:
from src.tools import to_ranges, count_im, xy, Circ
from matplotlib import pyplot as plt
from json import load
import numpy as np
import math as M
import cv2 as C

data = load(open('data.txt', 'r'))

Y_AXIS = int(data['Y'])
BEAD_FREQ = int(data['f'])
B_POS = (Y_AXIS, 700)
B_SIZ = 25

BEAD_SIZE = float(data['R'])
IBD = float(data['D'])

hough = lambda img, **kwargs: np.uint16(np.around(C.HoughCircles(
  img, method=C.HOUGH_GRADIENT, dp=1, **kwargs
)))

In [ ]:
lums = []
cache = {}

for i in range(1, count_im('out_dir') + 1):
  name = f'{i:03d}'
  img = C.imread(f'./out_dir/{name}.png', C.IMREAD_GRAYSCALE)

  lum = np.sum(img > 200)
  lums.append(lum)
  cache[name] = lum
mean, std = np.mean(lums), np.std(lums)

# delete items in cache that are not in lums
for k in list(cache.keys()):
  if cache[k] < mean + std:
    del cache[k]

ranges = to_ranges(list(map(int, cache.keys())))
MRS = np.mean([r[-1] - r[0] + 1 for r in ranges])
maxes = {}
for r in ranges:
  argmax = max(r, key=lambda k: cache[f"{k:03d}"])
  maxes[argmax] = cache[f"{argmax:03d}"]

print(MRS, ranges)
print(maxes)

img = C.imread(f'./in_dir/{max(maxes, key=maxes.get):03d}.png', C.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
REV = round(1.5 * BEAD_FREQ * BEAD_SIZE /(IBD + 2*BEAD_SIZE))

bead = max(maxes, key=lambda k: maxes[k]) - REV
if bead <= 0:
  bead = 1
print(f"Using frame {bead}")
img_ref = C.imread(f'./in_dir/{bead:03d}.png', C.IMREAD_GRAYSCALE)

x0, y0, x1, y1 = xy((B_POS[0], round(B_POS[1] - 25)), B_SIZ)
img_bead = img_ref[x0:x1, y0:y1]
bf = round((IBD + 2) * BEAD_SIZE)
img_base = img_ref[x0:x1, y0-bf:y1-bf]

In [ ]:
img_base2, img_bead2 = Circ(img_base, BEAD_SIZE), Circ(img_bead, BEAD_SIZE)

fig, axs = plt.subplots(1, 2)
axs[0].imshow(img_base2, cmap='gray')
axs[1].imshow(img_bead2, cmap='gray')
plt.show()

# add blur to both images
img_base2 = C.GaussianBlur(img_base2, (3, 3), 1)
img_bead2 = C.GaussianBlur(img_bead2, (3, 3), 1)

# plot diff
diff = img_base2 - img_bead2
diff = np.where(diff < 0.68*255, diff, 0)

s = 3

max_diff = 0
mdp = (0, 0)
for i in range(diff.shape[0] - s):
  for j in range(diff.shape[1] - s):
    if np.median(diff[i:i+s, j:j+s]) > max_diff:
      max_diff = np.median(diff[i:i+s, j:j+s])

      max_x = round(np.argmax(diff[i:i+s, j:j+s]) / s)
      max_y = np.argmax(diff[i:i+s, j:j+s]) % s
      mdp = (i + max_x, j + max_y)
# we can just use openCV's findContour

cx, cy = round(img_base2.shape[0] / 2), round(img_base2.shape[1] / 2)
plt.imshow(diff, cmap='gray')
slope = (mdp[0] - cx) / (mdp[1] - cy + 1e-6)
slope = round((180-( M.atan(slope) * 180 / M.pi)), 2)
plt.plot([cy, mdp[1]], [cx, mdp[0]], 'r')

plt.plot(cx, cy, 'ro')
plt.plot(mdp[1], mdp[0], 'bo')
plt.title(f"Slope: {slope}deg")

plt.show()

D = M.sqrt((cx - mdp[0])**2 + (cy - mdp[1])**2)
print(f"On circ of R = {BEAD_SIZE}, D = {IBD}")
print(f"(cx, cy) = ({cx}, {cy}), bead(x, y) = {mdp}, dist={D:.1f}")